# Solana Early Trader PNL Analysis

In [1]:
# Install necessary libraries
!pip install requests pandas python-dotenv



[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [152]:

import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import joblib
import aiohttp
import asyncio
from typing import List, Dict, Any


CACHE_FILE = "sol_price_cache.pkl"

MORALIS_API_KEY = os.environ.get("MORALIS_API_KEY")
HELIUS_API_KEY = os.environ.get("HELIUS_API_KEY")
COINGECKO_API_KEY = os.environ.get("GECKO_API")
HELIUS_RPC_URL = f"https://mainnet.helius-rpc.com/?api-key={HELIUS_API_KEY}"

# How many hours after a token's first trade to be considered "early"
EARLY_TRADING_WINDOW_HOURS = 6

# Minimum USD value of the first buy to qualify as an early trader
MINIMUM_INITIAL_BUY_USD = 100

# Minimum number of profitable trades for a wallet to appear in the final results
MIN_PROFITABLE_TRADES = 1


In [ ]:
def _safe_float(val):
    try:
        return float(val)
    except (TypeError, ValueError):
        return None

def get_solana_dex_trades(token_addresses: list, limit: int = 100) -> pd.DataFrame:
    """
    Fetch all DEX trades from Moralis for Solana token addresses.
    Uses cursor-based pagination until no more trades are available.

    Args:
        token_addresses (list): List of token mint addresses (max 3 at a time).
        limit (int): Number of results per request (max 100 supported by API).

    Returns:
        pd.DataFrame: Normalized trades DataFrame.
    """
    if not isinstance(token_addresses, list):
        raise TypeError("token_addresses must be a list of token mint addresses.")

    if not token_addresses:
        print("No token addresses provided. Returning empty DataFrame.")
        return pd.DataFrame()

    if len(token_addresses) > 3:
        raise ValueError("A maximum of 3 token addresses is allowed per request.")

    if not MORALIS_API_KEY:
        raise RuntimeError("MORALIS_API_KEY not set in environment variables.")

    headers = {
        "Accept": "application/json",
        "X-API-Key": MORALIS_API_KEY
    }

    all_trades = []

    for address in token_addresses:
        url = f"https://solana-gateway.moralis.io/token/mainnet/{address}/swaps"
        cursor = None
        print(f"Fetching ALL swaps for token: {address}...")

        while True:
            params = {"limit": limit, "order": "DESC", "transactionTypes": "buy,sell"}
            if cursor:
                params["cursor"] = cursor

            try:
                resp = requests.get(url, headers=headers, params=params, timeout=10)
                resp.raise_for_status()
                data = resp.json()
            except Exception as e:
                print(f"Error fetching trades for {address}: {e}")
                break

            trades = data.get("result", [])
            if not trades:
                break

            for trade in trades:
                direction = trade.get("transactionType")  # "buy" or "sell"
                bought = trade.get("bought", {}) or {}
                sold = trade.get("sold", {}) or {}

                amount_usd = _safe_float(trade.get("totalValueUsd"))
                token_bought_amt = _safe_float(bought.get("amount")) if direction == "buy" else _safe_float(sold.get("amount"))

                price_at_trade = None
                if amount_usd and token_bought_amt:
                    try:
                        price_at_trade = amount_usd / token_bought_amt
                    except ZeroDivisionError:
                        price_at_trade = None

                all_trades.append({
                    "block_time": pd.to_datetime(trade.get("blockTimestamp")),
                    "trader_id": trade.get("walletAddress"),
                    "token_bought_mint_address": bought.get("address") if direction == "buy" else sold.get("address"),
                    "token_bought_amount": token_bought_amt,
                    "token_sold_mint_address": sold.get("address") if direction == "buy" else bought.get("address"),
                    "token_sold_amount": _safe_float(sold.get("amount")) if direction == "buy" else _safe_float(bought.get("amount")),
                    "amount_usd": amount_usd,
                    "price_usd_at_trade": price_at_trade
                })

            cursor = data.get("cursor")
            if not cursor:
                break

            time.sleep(0.2)  # avoid hitting Moralis rate limit

        print(f"✅ Finished fetching {len(all_trades)} trades for token {address}")

    if not all_trades:
        print("No trades fetched. Returning empty DataFrame.")
        return pd.DataFrame()

    trades_df = pd.DataFrame(all_trades).sort_values("block_time", ascending=False).reset_index(drop=True)
    return trades_df


In [143]:
tokens = ['9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump']
trades_df = get_solana_dex_trades(tokens)

Fetching ALL swaps for token: 9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump...
✅ Finished fetching 7927 trades for token 9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump


In [144]:
trades_df.head()

,block_time,trader_id,token_bought_mint_address,token_bought_amount,token_sold_mint_address,token_sold_amount,amount_usd,price_usd_at_trade
0,2025-09-19 18:12:27+00:00,99Dq8FBXdw5DvT6JrBmQFMPfEHEUYx686VDnP6Nik6VZ,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,2.696402e+06,So11111111111111111111111111111111111111112,0.073007,17.198734,0.000006
1,2025-09-19 17:54:49+00:00,JD6rVaerbyz6wjQ433nrw6bFTgFrp46MiYmi8EtUAfsG,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,1.054229e+04,So11111111111111111111111111111111111111112,0.000294,0.069606,0.000007
2,2025-09-19 17:54:49+00:00,Ax8W8mcFTyPyoeSwvVc4Qq5cPVdGgD1rBc7MGBy1bEvL,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,2.022661e+07,So11111111111111111111111111111111111111112,0.563685,133.546424,0.000007
3,2025-09-19 17:54:49+00:00,JD6rVaerbyz6wjQ433nrw6bFTgFrp46MiYmi8EtUAfsG,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,7.579222e+03,So11111111111111111111111111111111111111112,0.000223,0.050042,0.000007
4,2025-09-19 17:54:49+00:00,JD6rVaerbyz6wjQ433nrw6bFTgFrp46MiYmi8EtUAfsG,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,2.963063e+03,So11111111111111111111111111111111111111112,0.000087,0.019564,0.000007


### Get traders first buy

In [147]:
# Define base tokens (USDC, USDT, SOL)
BASE_TOKENS = {
    "So11111111111111111111111111111111111111112",  # SOL
    "Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB",  # USDT
    "EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v"   # USDC
}

print("Step 3: Finding each trader's first buy for each token...")

# ✅ A "buy" means trader sold a base token (SOL/USDC/USDT)
buy_trades = trades_df[
    trades_df["token_sold_mint_address"].isin(BASE_TOKENS)
].copy()

# Get the earliest buy per trader/token pair
trader_first_buy_details_df = (
    buy_trades.loc[
        buy_trades.groupby(['trader_id', 'token_bought_mint_address'])['block_time'].idxmin()
    ]
)

# Keep the relevant columns
trader_first_buy_details_df = trader_first_buy_details_df[[
    'trader_id', 
    'token_bought_mint_address', 
    'block_time',
    'token_bought_amount',
    'amount_usd',
    'price_usd_at_trade'
]].rename(columns={'block_time': 'first_personal_buy_time'})

# Rename for clarity
trader_first_buy_details_df['first_buy_usd_amount'] = trader_first_buy_details_df['amount_usd']

print("\nTrader First Buy Valued DataFrame (`trader_first_buy_details_df`):")
display(trader_first_buy_details_df[['trader_id', 'token_bought_mint_address', 'first_buy_usd_amount', 'price_usd_at_trade']].head())


Step 3: Finding each trader's first buy for each token...

Trader First Buy Valued DataFrame (`trader_first_buy_details_df`):


,trader_id,token_bought_mint_address,first_buy_usd_amount,price_usd_at_trade
1365,1213eSdLLVcsMHh2qvXEtAevj2o1Tx92EMnuEz5wsZ2p,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,11.273039,0.000040
4563,12665qtP2MfNuq1oJ9P1BUBWYu6ZrYhrJHDzfsM9gcmk,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,7.403598,0.000065
3581,12rVVZEDvdXcacV1jpkeNgvZLSmJa5i53mFr1rQL6PiB,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,25.194970,0.000082
4945,13Fsb8y9RWM5BTKAuoWmo1Fir7vnwwSp3Quw8JCkCa4h,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,44.863565,0.000099
5884,145Baf8VADkDK8LrY4EiPj68yoCS3MsaJqJRajDNFn58,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,123.378155,0.000058


In [153]:
print("Step 5: Identifying the Early Traders Cohort...")

# 1. Get the earliest trade time for each token (token launch time proxy)
token_first_trade_times_df = trades_df.groupby('token_bought_mint_address').agg(
    token_launch_time=('block_time', 'min')
).reset_index()

# 2. Join this back to the first buys data
merged_df = pd.merge(
    trader_first_buy_details_df,
    token_first_trade_times_df,
    on='token_bought_mint_address',
    how='left'
)

# 3. Calculate time since launch for each trader's first buy
merged_df['time_since_launch'] = (
    merged_df['first_personal_buy_time'] - merged_df['token_launch_time']
)

# Apply early trader conditions
is_early = merged_df['time_since_launch'] <= timedelta(hours=EARLY_TRADING_WINDOW_HOURS)
is_min_buy_met = merged_df['first_buy_usd_amount'] >= MINIMUM_INITIAL_BUY_USD

# 4. Filter to get our final cohort
early_traders_df = merged_df[is_early & is_min_buy_met].copy()
early_traders_cohort = early_traders_df['trader_id'].unique()

print(f"\nIdentified {len(early_traders_cohort)} unique early traders.")
print("Sample Early Traders Cohort:", early_traders_cohort[:5])  # print first 5


Step 5: Identifying the Early Traders Cohort...

Identified 564 unique early traders.
Sample Early Traders Cohort: ['145Baf8VADkDK8LrY4EiPj68yoCS3MsaJqJRajDNFn58'
 '21PCJzbv9n4gLNwe6ZEDA7JPfmxqRWjrDHBkkx99GD75'
 '23Ttm8gnNGzAa6r9tFUDDqnP27C1kAZrodmCLLMyySd9'
 '23bDpxDVNyuVRMhFDBjfH89nbbEknewCeKfm35TWCL8Q'
 '243xxopK34M86JURN7L4CgCsPBxjFTHZXiG5rCyjKTdU']


In [154]:
print("Calculating lifetime metrics for the early trader cohort...")

# Filter all trades to only those made by our cohort
cohort_trades_df = trades_df[trades_df['trader_id'].isin(early_traders_cohort)].copy()

# --- Step 6: Acquisition Metrics ---
# Consider all trades where the trader bought tokens
buys_df = cohort_trades_df[cohort_trades_df['token_bought_amount'] > 0].copy()
# Use USD value from Moralis directly
buys_df['buy_usd_spent'] = buys_df['amount_usd']

trader_token_acquisition_df = buys_df.groupby(
    ['trader_id', 'token_bought_mint_address']
).agg(
    total_usd_spent=('buy_usd_spent', 'sum'),
    total_tokens_bought=('token_bought_amount', 'sum')
).reset_index()

trader_token_acquisition_df['avg_buy_price_usd'] = (
    trader_token_acquisition_df['total_usd_spent'] / trader_token_acquisition_df['total_tokens_bought']
)

# --- Step 8: Sales & Realized Profit ---
sells_df = cohort_trades_df[cohort_trades_df['token_sold_amount'] > 0].copy()
sells_df['sell_revenue_usd'] = sells_df['amount_usd']

# Join with acquisition cost to calculate realized profit
sells_df = pd.merge(
    sells_df,
    trader_token_acquisition_df[['trader_id', 'token_bought_mint_address', 'avg_buy_price_usd']],
    left_on=['trader_id', 'token_sold_mint_address'],
    right_on=['trader_id', 'token_bought_mint_address'],
    how='left'
)

sells_df['cost_of_goods_sold'] = sells_df['token_sold_amount'] * sells_df['avg_buy_price_usd']
sells_df['realized_pnl_usd'] = sells_df['sell_revenue_usd'] - sells_df['cost_of_goods_sold']

trader_token_sales_profit_df = sells_df.groupby(
    ['trader_id', 'token_sold_mint_address']
).agg(
    total_sales_revenue_usd=('sell_revenue_usd', 'sum'),
    realized_profit_usd=('realized_pnl_usd', 'sum')
).reset_index()

# --- Step 9: Combine Metrics ---
trader_token_metrics_df = trader_token_acquisition_df.rename(
    columns={'token_bought_mint_address': 'mint_address'}
)

trader_token_metrics_df = pd.merge(
    trader_token_metrics_df,
    trader_token_sales_profit_df.rename(columns={'token_sold_mint_address': 'mint_address'}),
    on=['trader_id', 'mint_address'],
    how='left'
).fillna(0)

print("\nCombined Trader-Token Metrics DataFrame:")
display(trader_token_metrics_df.head())


Calculating lifetime metrics for the early trader cohort...

Combined Trader-Token Metrics DataFrame:


,trader_id,mint_address,total_usd_spent,total_tokens_bought,avg_buy_price_usd,total_sales_revenue_usd,realized_profit_usd
0,145Baf8VADkDK8LrY4EiPj68yoCS3MsaJqJRajDNFn58,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,244.439610,4.284772e+06,0.000057,0.0,0.0
1,21PCJzbv9n4gLNwe6ZEDA7JPfmxqRWjrDHBkkx99GD75,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,549.008080,1.721967e+07,0.000032,0.0,0.0
2,23Ttm8gnNGzAa6r9tFUDDqnP27C1kAZrodmCLLMyySd9,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,731.466671,2.675450e+07,0.000027,0.0,0.0
3,23bDpxDVNyuVRMhFDBjfH89nbbEknewCeKfm35TWCL8Q,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,821.022066,1.402908e+07,0.000059,0.0,0.0
4,243xxopK34M86JURN7L4CgCsPBxjFTHZXiG5rCyjKTdU,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,714.773351,6.443539e+07,0.000011,0.0,0.0


In [155]:
# ---------- Helpers ----------

async def fetch_solana_rpc(payload: dict) -> dict:
    """Make a raw RPC call to Helius."""
    async with aiohttp.ClientSession() as session:
        async with session.post(HELIUS_RPC_URL, json=payload, timeout=30) as resp:
            resp.raise_for_status()
            return await resp.json()

async def fetch_dexscreener_price(mint: str) -> float:
    """Fetch priceUsd for a token from Dexscreener."""
    url = f"https://api.dexscreener.com/latest/dex/tokens/{mint}"
    async with aiohttp.ClientSession() as session:
        async with session.get(url, timeout=10) as resp:
            if resp.status != 200:
                return None
            data = await resp.json()
    pairs = data.get("pairs") or []
    if not pairs:
        return None
    return float(pairs[0].get("priceUsd", 0)) if pairs[0].get("priceUsd") else None

# ---------- Core Function ----------

async def get_current_balances_and_prices(traders: List[str], token_mints: List[str]) -> pd.DataFrame:
    """
    Fetch SOL balance + SPL token balances for each trader, 
    enrich with Dexscreener price, compute USD value.
    """
    all_rows = []

    token_mints_set = set([m.strip() for m in token_mints])  # normalize

    for trader in traders:
        # 1️⃣ SOL balance
        sol_payload = {"jsonrpc": "2.0", "id": 1, "method": "getBalance", "params": [trader]}
        sol_res = await fetch_solana_rpc(sol_payload)
        sol_balance = sol_res.get("result", {}).get("value", 0) / 1e9  # lamports → SOL

        # 2️⃣ Token balances
        tokens_payload = {
            "jsonrpc": "2.0",
            "id": "1",
            "method": "getTokenAccountsByOwner",
            "params": [
                trader,
                {"programId": "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA"},
                {"encoding": "jsonParsed"}
            ]
        }
        tok_res = await fetch_solana_rpc(tokens_payload)
        accounts = tok_res.get("result", {}).get("value", []) or []

        # 3️⃣ Build a map of balances for all relevant tokens (even zero)
        balances_map = {mint: 0.0 for mint in token_mints_set}

        for acc in accounts:
            info = acc.get("account", {}).get("data", {}).get("parsed", {}).get("info", {})
            mint = info.get("mint", "").strip()
            if mint in token_mints_set:
                token_amount = info.get("tokenAmount", {})
                ui_amount = token_amount.get("uiAmount", 0.0)
                balances_map[mint] = float(ui_amount)

        # 4️⃣ Fetch Dexscreener prices concurrently (optional: can batch)
        for mint, balance in balances_map.items():
            price = await fetch_dexscreener_price(mint) or 0.0
            usd_value = balance * price
            all_rows.append({
                "trader_id": trader,
                "sol_balance": sol_balance,
                "mint_address": mint,
                "token_balance": balance,
                "current_price": price,
                "current_value_usd": usd_value
            })

    balances_df = pd.DataFrame(all_rows)
    return balances_df


In [ ]:
# traders = [
#     "ABxm8x2UaMjyNamw4YeS3xLeNbXqDAUypi4APTQwWtGE",
#     "BtDaZUqHr2mKH5EYQCztuerHBuBEfQNYdquTDtEZp2Ym"
# ]
# mints = [
#     "iD9FqgeKmp58a4pTvh45MtKkHkudKNptFRdR8m6pump"
# ]

In [ ]:
# --- Usage example ---
# balances_df = await get_current_balances_and_prices(early_traders_cohort, tokens)


In [ ]:
# balances_df

,trader_id,sol_balance,mint_address,token_balance,current_price,current_value_usd
0,2Eui92G64bACkivCZdKyWahSfiHacJe981aAM1n6bEki,18.324865,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
1,3pEtH5oaLRgGoLgw1qbvNriGZWYEL5NzUqQgwcAR7c8G,84.394283,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
2,5oznJEeZbTeFX71qHWzftT4pJCiaDJypSTZfBCH4fWaB,11.484231,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
3,6MTrnuMuSwTHzJTv8JYVRvaqTgq4fKZJBqyyf3FhVff3,0.067169,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
4,85xHFu8h9HMBELSHpetXeCQbzYjD6v5k1DpwbMecz5PZ,176.326988,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
5,8N8Jztzf9PFhttqjtqwqT8bbdU6JB6xu4A4FP8qvZbvX,0.807856,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
6,8TmEdDjLkwpWvNKTZtTtJcHjy5MuPbaGq1toY41WNCaf,1.935775,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
7,8fSnLTnRViK83dDesivTsPx2wiRhwti9xoafeMGjEyLJ,47.201341,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
8,9WbCX6bUE9YKDtXe2rW7bqocekNkmFTGSkvDWVAp63Pb,11.224389,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,0.000000e+00,0.000007,0.000000e+00
9,BNahnx13rLru9zxuWNGBD7vVv1pGQXB11Q7qeTyupdWf,118.358593,9zdV7hSbisPRfALAEosvRT9A6ZhMrGoUnEJ4738Mpump,3.548162e+07,0.000007,2.418072e+02


In [156]:
# --- Fetch balances ---
balances_df = await get_current_balances_and_prices(
    traders=early_traders_cohort, 
    token_mints=list(tokens)
)

# --- Merge with trader-token metrics ---
trader_token_metrics_df = pd.merge(
    trader_token_metrics_df,
    balances_df[['trader_id', 'mint_address', 'token_balance', 'current_price', 'current_value_usd']],
    on=['trader_id', 'mint_address'],
    how='left'
).fillna(0)

# --- Unrealized PNL ---
trader_token_metrics_df['unrealized_cost_basis'] = trader_token_metrics_df['token_balance'] * trader_token_metrics_df['avg_buy_price_usd']
trader_token_metrics_df['estimated_unrealised_pnl'] = trader_token_metrics_df['current_value_usd'] - trader_token_metrics_df['unrealized_cost_basis']

# --- Total PNL per token ---
trader_token_metrics_df['token_total_pnl'] = trader_token_metrics_df['realized_profit_usd'] + trader_token_metrics_df['estimated_unrealised_pnl']
trader_token_metrics_df['is_token_profitable'] = (trader_token_metrics_df['token_total_pnl'] > 0).astype(int)

# --- Aggregate to trader level ---
final_summary_df = trader_token_metrics_df.groupby('trader_id').agg(
    overall_total_usd_spent=('total_usd_spent', 'sum'),
    overall_total_sales_revenue=('total_sales_revenue_usd', 'sum'),
    overall_realized_profit_usd=('realized_profit_usd', 'sum'),      # ✅ fixed name
    overall_estimated_unrealised_pnl=('estimated_unrealised_pnl', 'sum'),
    overall_current_value_of_holdings_usd=('current_value_usd', 'sum'),
    overall_total_pnl=('token_total_pnl', 'sum'),
    num_unique_tokens_traded=('mint_address', 'nunique'),
    num_tokens_in_profit=('is_token_profitable', 'sum')
).reset_index()


final_summary_df['ROI'] = (final_summary_df['overall_total_pnl'] / final_summary_df['overall_total_usd_spent']).fillna(0)
final_summary_df['win_rate'] = (final_summary_df['num_tokens_in_profit'] / final_summary_df['num_unique_tokens_traded']).fillna(0)

# --- Merge trade counts ---
trade_counts = cohort_trades_df['trader_id'].value_counts().reset_index()
trade_counts.columns = ['trader_id', 'number_of_trades']
final_summary_df = pd.merge(final_summary_df, trade_counts, on='trader_id', how='left')

# --- Filter & Sort ---
final_summary_df = final_summary_df[final_summary_df['num_tokens_in_profit'] >= MIN_PROFITABLE_TRADES]
final_summary_df = final_summary_df.sort_values(by='ROI', ascending=False)

# --- Display top 20 traders ---
print("\n--- 🏆 Final Early Trader Profitability Report ---")
display(final_summary_df[[
    'trader_id',
    'ROI',
    'overall_total_pnl',
    'overall_total_usd_spent',
    'overall_current_value_of_holdings_usd',
    'win_rate',
    'number_of_trades'
]].head(20))



--- 🏆 Final Early Trader Profitability Report ---


,trader_id,ROI,overall_total_pnl,overall_total_usd_spent,overall_current_value_of_holdings_usd,win_rate,number_of_trades
